<a href="https://colab.research.google.com/github/Usama-zia/openai_prompting_demo/blob/main/use_cases_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Libraries

In [1]:
!pip install openai==0.28
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.3 MB/s eta 0:00:00


## Prep the Environment

In [2]:
# import libraries
import PyPDF2
import re
import os
import openai
import spacy
import nltk
import re
from nltk import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# Please use your own API key here.
API_KEY = ""

##Use Case 1 (Implemented)



* From a filled contract what could be the process to convert it as a template

###Solution


*   Step 1: Using function calling to get the function for "convert_document_to_template()"
*   Step 2: call the function returned from function calling.
*   Step 3: Read the pdf using PyPDF2 library.
*   Step 4: Use Spacy and NLTK to filter out proper Noun Specifically names of Persons(e.g, Will Smith)
*   Step 5: Replace Filtered names with placeholder '[Name]'.
*   Step 6: Use pattern recognition (re -> regex), to filter out addresses of people and replace them with placeholder '[Address]'.
*   Step 7: Use pattern recognition (re -> regex), to filter out dates and replace them with placeholder '[Dates]'.
*   Step 8: display the document text converted to template.



###Fucntions for Document/Text Manipulation.

In [10]:
# This function will read PDF file page by page and return the text.
def read_pdf(file_address):
  reader = PyPDF2.PdfReader(file_address)
  for i in range(len(reader.pages)):
    page = reader.pages[i]
    text = page.extract_text()
    return text

# This function will filter proper nouns/ people names and replace them with placeholder.
def replace_people_names_with_placeholders(text):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)

  names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
  filtered_names = []
  name_replaced_text = ""
  for name in names:
    words = word_tokenize(name)
    tagged_words = pos_tag(words)
    if all(tag == 'NNP' for word, tag in tagged_words):
      filtered_names.append(name)

  filtered_names = list(set(filtered_names))
  for filtered_name in filtered_names:
    text = re.sub(filtered_name, '[Name]', text)

  return text

#This function will replace addresses with placeholder.
def replace_address_with_placeholder(text):
  address_pattern = r"\d+\s+[a-zA-Z\s]+,\s*[a-zA-Z\s]+,\s*[a-zA-Z\s]+"
  address_match = re.findall(address_pattern, text)
  for address in address_match:
    text = re.sub(address, '[address]', text)
  return text

# This function will replace dates with placeholders.
def replace_dates_with_placeholder(text):
  pattern = r'\b\d{2}-\d{2}-\d{4}\b'
  dates_matched = re.findall(pattern, text)
  for date in dates_matched:
    text = re.sub(date, '[date]', text)
  return text

###Functions used by Function Calling.

In [5]:
#This function will be called using the function Calling.
def convert_document_to_template(str = None):
  print('ChatGPT function Calling successfull.')
  ## accessing document from google drive.
  use_case_1_pdf = "/content/drive/MyDrive/use_case_1.pdf"
  pdf_text = read_pdf(use_case_1_pdf)
  temp_text = replace_people_names_with_placeholders(pdf_text)
  temp_text = replace_address_with_placeholder(temp_text)
  temp_text = replace_dates_with_placeholder(temp_text)
  print(temp_text)

#This function is used to test if function calling is working properly.
def dummy_test_function():
  print("Dummy function has been called.")
  return 'dummy function calling finished.'

In [6]:
#define model parameters and system prompt
llm_model = "gpt-3.5-turbo"
llm_system_prompt = "You are an assistant that, helps me update different documents using function calling."
openai.api_key = (API_KEY)

### Providing details of functions for function calling to Openai Api using, JSON.

In [7]:
# define the function with its parameters for calling
signature_convert_document_to_template = {
    "name": "convert_document_to_template",
    "description": "convert the given master document to template.",
    "parameters": {
        "type": "object",
        "properties": {
        },
        "required": [],
    }
}
#Define dummy function for testing
signature_dummy_test_function = {
    "name": "dummy_test_function",
    "description": "to check if fucntion calling works given two fucntions,these are test string, convert, update, template, document.",
    "parameters": {
        "type": "object",
        "properties": {
        },
        "required": [],
    }
}

####This function will handle the API calling, and fucntion calling.

In [8]:
def function_calling_handler(messages, function_call: str = "auto"):
    """Fetch completion from OpenAI's GPT"""
    print('Working...')
    res = openai.ChatCompletion.create(
        model=llm_model,
        messages=messages,
        functions=[signature_convert_document_to_template, signature_dummy_test_function],
        function_call=function_call
    )

    response = res["choices"][0]["message"]

    # call functions requested by the model
    if response.get("function_call"):
        function_name = response["function_call"]["name"]
        if function_name == "convert_document_to_template":
            #args = json.loads(response["function_call"]["arguments"])
            headlines = convert_document_to_template()
        if function_name == "dummy_test_function":
            headlines = convert_document_to_template()


#####Sending query to the API and displaying the text converted to template.

In [11]:
messages = []
messages.append({"role": "system", "content": llm_system_prompt})
messages.append({"role": "user", "content": 'convert my master document to template.'})
function_calling_handler(messages)

Working...
ChatGPT function Calling successfull.
 
This Agreement is made and entered into as of [date] , by and between:  
[Name] , residing at  [address], here  in after referred to as Party A,  
and [Name] , residing at  [address], here  in after referred to as Party 
B. 
Whereas , Party A and Party B desire to agree  to define the terms and conditions of their 
relationship . 
Now, Therefore,  in consideration of the mutual promises, covenants, and agreements 
contained herein, and for other good and valuable consi deration, the receipt and sufficiency 
of which are hereby acknowledged, the parties hereto agree as follows:  
Purpose of Agreement: The purpose of this Agreement is to define the terms and conditions 
of the relationship between Party A and Party B.  
Scope of  Services: Party A agrees to provide services to Party B, and Party B agrees to 
compensate Party A for these services as per the terms outlined in this Agreement.  
Term: This Agreement shall commence on [date] 

##Use Case 2 (Proposed Solutions)


*    What could be the process to update the master document from a user query such as : "Add an article between 6 and 7 about privacy law"

### Solution 1 (Using PyPDF2 library)

*   Step 1: Using function calling to get the function for "add_to_document_and_update()".
*   Step 2: call the function returned from function calling.
*   Step 3: Read both pdfs using PyPDF2 library.
*   Step 4: Iterate through the pages of the PDF document using the '.pages[page_number]' method.
*   Step 5: Use the `reportlab` library to generate a 'Canvas' object for each page.
*   Step 6: find coordinates for Article 6 and 7 on the canvas.
*   Step 7: convert Article that will go between 6 and 7 to canvas.
*   Step 8: Add text between those coordinates at the canvas.
*   Step 9: Merge the changes using 'add_page()' method of the 'PdfWriter'. object.
*   Step 10: use regex to find pattern of numbers for headings and bullet points.
*   Step 11: Rearrange and sort bullet points and heading numbers accordingly.

### Solution 2 (Using OCR(Optical Character Recognition))
*   Step 1: Read both pdfs using library such as tessarect and convert them to images for processing.
*   Step 2: Get coordinates of Article 6's start and end points.
*   Step 3: Get coordinates of Article 7's start.
*   Step 4: Add new Article at the End of Atricale 6 and the start of Article 7, and update all the coordinates accordingly.
*   Step 5: take all the data from the start of Article 6 to the end of Article 7 and rearrange any heading and bullet point numbers accordingly.
*   Step 6: convert image back into accessable pdf after Adding the Article.
*   Step 7: Return the updated pdf.

### Solution 3 (Using Json)
*   Step 1: Read the pdfs.
*   Step 2: convert them both to pdf.
*   Step 3: Find the article 6 and 7.
*   Step 4: add the new Article between the article 6 and 7.
*   Step 5: Rearragne heading and bullet point numbers by asscessing keys for content.
*   Step 6: convert finished json back to pdf.
*   Step 7: Return the updated pdf.

## Conclusion

I have tried to implement these use cases on the server side without exposing personal information to Chatgpt due to concerns for data privacy laws, even though use case 1 can be solved at least partially by parsing the master document to Chatgpt. This is why chatgpt will only return a function that needs to be called, but no parameters. However, parameters can be provided in the prompt and function calling will return the appropriate parameters.

For use case 1, solution is not complete, as it will only work for one type of date, or address format, and will not filter names that have middle names. A detailed pattern recognition system/class will need to be implemented to identify all different date, address, names of people, places, countries, phone numbers or other related patterns, so that a document can be accurately turned into a template.

For use case 2, I have provided three possible solutions, one or all three can be implemented depending upon the situation, for example, if the master document was scanned using an image scanner, then we can implement solution 2.


## Suggentions

These function calls, and other api callings can be used to gather data, which can help to recognise different KPI's, monitor performance and determine costs
which can help with pricing.

To gather and utilize the data effectively, A new database schema can be created to store data used purely for data analysis.